# AURA V14 PHOENIX - Multi-Task Toxicity Detection

**Architecture**: RoBERTa + Task-Specific Attention + 4 Classification Heads

**Tasks**:
1. **Toxicity Detection** (Binary: Toxic/Non-Toxic)
2. **Emotion Recognition** (Multi-label: 7 Ekman emotions)
3. **Sentiment Analysis** (Binary: Positive/Negative)
4. **Reporting Detection** (Binary: Direct/Reported speech)

**Key Innovation**: Each task has its own attention layer that learns WHERE to look in the sequence.

---

In [ ]:
# Cell 1: Imports & Setup
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torch.optim.lr_scheduler import OneCycleLR
from transformers import RobertaModel, RobertaTokenizer
from tqdm.notebook import tqdm
from sklearn.metrics import f1_score, classification_report
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

In [ ]:
# Cell 2: Configuration V14 (PHOENIX)
CONFIG = {
    'encoder': 'roberta-base',
    'max_length': 128,
    'num_emotion_classes': 7,
    'dropout': 0.3,
    'weight_decay': 0.01,
    'batch_size': 16,
    'gradient_accumulation': 4,
    'epochs': 5,
    'lr_encoder': 2e-5,
    'lr_heads': 5e-5,
    'warmup_ratio': 0.1,
    'patience': 3,
    'focal_gamma': 2.0,
    'label_smoothing': 0.1,
    'freezing_epochs': 1,
}

EMO_COLS = ['anger', 'disgust', 'fear', 'joy', 'sadness', 'surprise', 'neutral']
DATA_DIR = '/kaggle/input/aura-v14-data'
print("V14 PHOENIX Loaded: RoBERTa + Task-Specific Attention 🔥")

In [ ]:
# Cell 3: Task-Specific Attention Layer
class TaskAttention(nn.Module):
    """Each task learns WHERE to look in the sequence."""
    def __init__(self, hidden_size):
        super().__init__()
        self.query = nn.Linear(hidden_size, hidden_size)
        self.key = nn.Linear(hidden_size, hidden_size)
        self.value = nn.Linear(hidden_size, hidden_size)
        self.scale = hidden_size ** 0.5
        
    def forward(self, hidden_states, attention_mask):
        # hidden_states: [batch, seq_len, hidden]
        # Use [CLS] token (position 0) as query
        cls_token = hidden_states[:, 0:1, :]  # [batch, 1, hidden]
        
        Q = self.query(cls_token)      # [batch, 1, hidden]
        K = self.key(hidden_states)    # [batch, seq, hidden]
        V = self.value(hidden_states)  # [batch, seq, hidden]
        
        # Attention scores
        scores = torch.bmm(Q, K.transpose(1, 2)) / self.scale  # [batch, 1, seq]
        
        # Mask padding tokens
        if attention_mask is not None:
            mask = attention_mask.unsqueeze(1)  # [batch, 1, seq]
            scores = scores.masked_fill(mask == 0, float('-inf'))
        
        attn_weights = F.softmax(scores, dim=-1)  # [batch, 1, seq]
        
        # Weighted sum of values
        output = torch.bmm(attn_weights, V)  # [batch, 1, hidden]
        return output.squeeze(1), attn_weights.squeeze(1)

print('TaskAttention layer defined.')

In [ ]:
# Cell 4: AURA PHOENIX Model
class AURA_PHOENIX(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.roberta = RobertaModel.from_pretrained(config['encoder'])
        hidden = self.roberta.config.hidden_size  # 768 for roberta-base
        
        # Task-Specific Attention Layers
        self.tox_attention = TaskAttention(hidden)
        self.emo_attention = TaskAttention(hidden)
        self.sent_attention = TaskAttention(hidden)
        self.report_attention = TaskAttention(hidden)
        
        # Dropout
        self.dropout = nn.Dropout(config['dropout'])
        
        # Classification Heads
        self.toxicity_head = nn.Linear(hidden, 2)
        self.emotion_head = nn.Linear(hidden, config['num_emotion_classes'])
        self.sentiment_head = nn.Linear(hidden, 2)
        self.reporting_head = nn.Linear(hidden, 1)  # Binary sigmoid
        
        # Bias Initialization for imbalanced toxicity (~5% toxic)
        with torch.no_grad():
            self.toxicity_head.bias[0] = 2.5   # Non-Toxic
            self.toxicity_head.bias[1] = -2.5  # Toxic
        
    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state  # [batch, seq, hidden]
        
        # Each task attends to different parts of the sequence
        tox_repr, tox_attn = self.tox_attention(hidden_states, attention_mask)
        emo_repr, emo_attn = self.emo_attention(hidden_states, attention_mask)
        sent_repr, sent_attn = self.sent_attention(hidden_states, attention_mask)
        report_repr, report_attn = self.report_attention(hidden_states, attention_mask)
        
        # Apply dropout
        tox_repr = self.dropout(tox_repr)
        emo_repr = self.dropout(emo_repr)
        sent_repr = self.dropout(sent_repr)
        report_repr = self.dropout(report_repr)
        
        return {
            'toxicity': self.toxicity_head(tox_repr),
            'emotion': self.emotion_head(emo_repr),
            'sentiment': self.sentiment_head(sent_repr),
            'reporting': self.reporting_head(report_repr).squeeze(-1),
            'attention': {
                'tox': tox_attn, 'emo': emo_attn, 
                'sent': sent_attn, 'report': report_attn
            }
        }

print('AURA PHOENIX model defined (RoBERTa + 4 Task Attention Heads).')

In [ ]:
# Cell 5: Loss Functions
def focal_loss(logits, targets, gamma=2.0, weight=None, smoothing=0.0):
    """Focal Loss with optional Label Smoothing."""
    ce = F.cross_entropy(logits, targets, weight=weight, reduction='none', label_smoothing=smoothing)
    p_t = torch.exp(-ce)
    return ((1 - p_t) ** gamma * ce).mean()

def compute_loss(outputs, batch, tox_weights, config):
    """Multi-task loss computation."""
    total_loss = torch.tensor(0.0, device=outputs['toxicity'].device)
    
    # 1. Toxicity Loss (Focal + Label Smoothing)
    if batch['task_mask_tox'].sum() > 0:
        tox_logits = outputs['toxicity'][batch['task_mask_tox']]
        tox_labels = batch['tox_label'][batch['task_mask_tox']]
        total_loss += focal_loss(tox_logits, tox_labels, 
                                 gamma=config['focal_gamma'], 
                                 weight=tox_weights,
                                 smoothing=config['label_smoothing'])
    
    # 2. Emotion Loss (BCE)
    if batch['task_mask_emo'].sum() > 0:
        emo_logits = outputs['emotion'][batch['task_mask_emo']]
        emo_labels = batch['emo_label'][batch['task_mask_emo']]
        total_loss += F.binary_cross_entropy_with_logits(emo_logits, emo_labels)
    
    # 3. Sentiment Loss (Focal)
    if batch['task_mask_sent'].sum() > 0:
        sent_logits = outputs['sentiment'][batch['task_mask_sent']]
        sent_labels = batch['sent_label'][batch['task_mask_sent']]
        total_loss += focal_loss(sent_logits, sent_labels, 
                                 gamma=config['focal_gamma'],
                                 smoothing=config['label_smoothing'])
    
    # 4. Reporting Loss (BCE)
    if batch['task_mask_report'].sum() > 0:
        report_logits = outputs['reporting'][batch['task_mask_report']]
        report_labels = batch['report_label'][batch['task_mask_report']].float()
        total_loss += F.binary_cross_entropy_with_logits(report_logits, report_labels)
    
    return total_loss

print('Loss functions defined.')

In [ ]:
# Cell 6: Dataset Classes
class BaseDataset(Dataset):
    def __init__(self, csv_path, tokenizer, max_len):
        self.df = pd.read_csv(csv_path)
        self.tokenizer = tokenizer
        self.max_len = max_len
    def __len__(self): return len(self.df)
    def encode(self, text):
        return self.tokenizer.encode_plus(
            str(text), max_length=self.max_len, 
            padding='max_length', truncation=True, 
            return_tensors='pt'
        )

class ToxicityDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(int(row['label']), dtype=torch.long),
            'emo_label': torch.zeros(7),
            'sent_label': torch.tensor(-1, dtype=torch.long),
            'report_label': torch.tensor(-1, dtype=torch.long),
            'task': 'toxicity'
        }

class EmotionDataset(BaseDataset):
    def __init__(self, csv_path, tokenizer, max_len, emo_cols):
        super().__init__(csv_path, tokenizer, max_len)
        self.emo_cols = emo_cols
        if 'label_sum' in self.df.columns:
            self.df = self.df[self.df['label_sum'] > 0].reset_index(drop=True)
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long),
            'emo_label': torch.tensor([float(row[c]) for c in self.emo_cols]),
            'sent_label': torch.tensor(-1, dtype=torch.long),
            'report_label': torch.tensor(-1, dtype=torch.long),
            'task': 'emotion'
        }

class SentimentDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long),
            'emo_label': torch.zeros(7),
            'sent_label': torch.tensor(int(row['label']), dtype=torch.long),
            'report_label': torch.tensor(-1, dtype=torch.long),
            'task': 'sentiment'
        }

class ReportingDataset(BaseDataset):
    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        enc = self.encode(row['text'])
        return {
            'input_ids': enc['input_ids'].flatten(),
            'attention_mask': enc['attention_mask'].flatten(),
            'tox_label': torch.tensor(-1, dtype=torch.long),
            'emo_label': torch.zeros(7),
            'sent_label': torch.tensor(-1, dtype=torch.long),
            'report_label': torch.tensor(int(row['is_reporting']), dtype=torch.long),
            'task': 'reporting'
        }

def collate_fn(batch):
    tasks = [x['task'] for x in batch]
    return {
        'input_ids': torch.stack([x['input_ids'] for x in batch]),
        'attention_mask': torch.stack([x['attention_mask'] for x in batch]),
        'tox_label': torch.stack([x['tox_label'] for x in batch]),
        'emo_label': torch.stack([x['emo_label'] for x in batch]),
        'sent_label': torch.stack([x['sent_label'] for x in batch]),
        'report_label': torch.stack([x['report_label'] for x in batch]),
        'task_mask_tox': torch.tensor([t == 'toxicity' for t in tasks], dtype=torch.bool),
        'task_mask_emo': torch.tensor([t == 'emotion' for t in tasks], dtype=torch.bool),
        'task_mask_sent': torch.tensor([t == 'sentiment' for t in tasks], dtype=torch.bool),
        'task_mask_report': torch.tensor([t == 'reporting' for t in tasks], dtype=torch.bool)
    }

print('Dataset classes defined.')

In [ ]:
# Cell 7: Load Data
tokenizer = RobertaTokenizer.from_pretrained(CONFIG['encoder'])

tox_train = ToxicityDataset(f'{DATA_DIR}/toxicity_train.csv', tokenizer, CONFIG['max_length'])
emo_train = EmotionDataset(f'{DATA_DIR}/emotions_train.csv', tokenizer, CONFIG['max_length'], EMO_COLS)
sent_train = SentimentDataset(f'{DATA_DIR}/sentiment_train.csv', tokenizer, CONFIG['max_length'])
report_train = ReportingDataset(f'{DATA_DIR}/reporting_examples.csv', tokenizer, CONFIG['max_length'])
tox_val = ToxicityDataset(f'{DATA_DIR}/toxicity_val.csv', tokenizer, CONFIG['max_length'])

train_set = ConcatDataset([tox_train, emo_train, sent_train, report_train])
train_loader = DataLoader(train_set, batch_size=CONFIG['batch_size'], shuffle=True, collate_fn=collate_fn, num_workers=2)
val_loader = DataLoader(tox_val, batch_size=CONFIG['batch_size'], shuffle=False, collate_fn=collate_fn)

tox_weights = torch.tensor([0.5, 2.0], device=device)  # Class weights for imbalance

print(f'Training samples: {len(train_set)}')
print(f'  - Toxicity: {len(tox_train)}')
print(f'  - Emotions: {len(emo_train)}')
print(f'  - Sentiment: {len(sent_train)}')
print(f'  - Reporting: {len(report_train)}')
print(f'Validation samples: {len(tox_val)}')

In [ ]:
# Cell 8: Training Functions (Following NB10 patterns)
def plot_history(history, metric):
    """Plot training history (from NB10)."""
    plt.figure(figsize=(10, 4))
    plt.plot(history[metric], label='Train')
    if f'val_{metric}' in history:
        plt.plot(history[f'val_{metric}'], label='Validation')
    plt.title(f'Model {metric.capitalize()}')
    plt.ylabel(metric.capitalize())
    plt.xlabel('Epoch')
    plt.legend()
    plt.grid(True)
    plt.show()

def train_epoch(model, loader, optimizer, scheduler, config, tox_weights, epoch):
    model.train()
    
    # Progressive Freezing (from V10)
    if epoch <= config['freezing_epochs']:
        print(f"❄️ Epoch {epoch}: Encoder FROZEN")
        for param in model.roberta.parameters():
            param.requires_grad = False
    else:
        print(f"🔥 Epoch {epoch}: Encoder UNFROZEN")
        for param in model.roberta.parameters():
            param.requires_grad = True
    
    total_loss = 0
    optimizer.zero_grad()
    pbar = tqdm(loader, desc=f'Epoch {epoch}')
    
    for step, batch in enumerate(pbar):
        for k, v in batch.items():
            if isinstance(v, torch.Tensor):
                batch[k] = v.to(device)
        
        outputs = model(batch['input_ids'], batch['attention_mask'])
        loss = compute_loss(outputs, batch, tox_weights, config)
        
        (loss / config['gradient_accumulation']).backward()
        
        if (step + 1) % config['gradient_accumulation'] == 0:
            nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
        
        total_loss += loss.item()
        pbar.set_postfix({'loss': f'{loss.item():.4f}'})
    
    return total_loss / len(loader)

@torch.no_grad()
def validate(model, loader):
    model.eval()
    preds, labels = [], []
    
    for batch in tqdm(loader, desc='Validating', leave=False):
        ids = batch['input_ids'].to(device)
        mask = batch['attention_mask'].to(device)
        
        out = model(ids, mask)
        preds.extend(out['toxicity'].argmax(1).cpu().numpy())
        labels.extend(batch['tox_label'].cpu().numpy())
    
    return f1_score(labels, preds, average='macro')

print('Training functions defined.')

In [ ]:
# Cell 9: Main Training Loop
print("="*60)
print("AURA V14 PHOENIX - TRAINING")
print("="*60)

model = AURA_PHOENIX(CONFIG).to(device)

# Optimizer with different LR for encoder vs heads (from V10)
optimizer = torch.optim.AdamW([
    {'params': model.roberta.parameters(), 'lr': CONFIG['lr_encoder']},
    {'params': model.tox_attention.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.emo_attention.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.sent_attention.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.report_attention.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.toxicity_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.emotion_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.sentiment_head.parameters(), 'lr': CONFIG['lr_heads']},
    {'params': model.reporting_head.parameters(), 'lr': CONFIG['lr_heads']},
], weight_decay=CONFIG['weight_decay'])

total_steps = len(train_loader) * CONFIG['epochs'] // CONFIG['gradient_accumulation']
scheduler = OneCycleLR(
    optimizer, 
    max_lr=[CONFIG['lr_encoder']] + [CONFIG['lr_heads']]*8,
    total_steps=total_steps, 
    pct_start=CONFIG['warmup_ratio']
)

# Training history (from NB10)
history = {'loss': [], 'val_f1': []}
best_val_f1 = 0.0
patience_counter = 0

for epoch in range(1, CONFIG['epochs'] + 1):
    avg_loss = train_epoch(model, train_loader, optimizer, scheduler, CONFIG, tox_weights, epoch)
    val_f1 = validate(model, val_loader)
    
    history['loss'].append(avg_loss)
    history['val_f1'].append(val_f1)
    
    print(f'Epoch {epoch}: Loss={avg_loss:.4f}, Val F1={val_f1:.4f}')
    
    if val_f1 > best_val_f1:
        best_val_f1 = val_f1
        patience_counter = 0
        torch.save(model.state_dict(), 'aura_v14_phoenix.pt')
        print('>>> NEW BEST MODEL SAVED <<<')
    else:
        patience_counter += 1
        if patience_counter >= CONFIG['patience']:
            print(f'Early stopping at epoch {epoch}')
            break

print(f"\nTraining Complete. Best Val F1: {best_val_f1:.4f}")

In [ ]:
# Cell 10: Plot Training History
plot_history(history, 'loss')
plot_history(history, 'val_f1')

In [ ]:
# Cell 11: Inference Function with Reporting Adjustment
@torch.no_grad()
def predict_toxicity(model, tokenizer, text, threshold=0.5):
    """Predict toxicity with reporting-aware adjustment."""
    model.eval()
    
    enc = tokenizer.encode_plus(
        text, max_length=128, padding='max_length', 
        truncation=True, return_tensors='pt'
    )
    
    ids = enc['input_ids'].to(device)
    mask = enc['attention_mask'].to(device)
    
    out = model(ids, mask)
    
    tox_probs = F.softmax(out['toxicity'], dim=1)
    tox_score = tox_probs[0, 1].item()  # P(toxic)
    
    report_score = torch.sigmoid(out['reporting']).item()
    
    # Adjust toxicity if reporting detected
    if report_score > 0.5:
        adjusted_tox = tox_score * (1 - report_score * 0.5)  # Reduce confidence
    else:
        adjusted_tox = tox_score
    
    is_toxic = adjusted_tox > threshold
    
    return {
        'text': text,
        'raw_toxicity': tox_score,
        'reporting_score': report_score,
        'adjusted_toxicity': adjusted_tox,
        'is_toxic': is_toxic
    }

print('Inference function defined.')

In [ ]:
# Cell 12: Stress Test
print("="*60)
print("STRESS TEST")
print("="*60)

# Load best model
model.load_state_dict(torch.load('aura_v14_phoenix.pt'))
model.eval()

test_cases = [
    # Should be NON-TOXIC (negative but not toxic)
    "I hate rainy Mondays",
    "This soup is disgusting",
    "I am so angry at the traffic",
    "This movie was terrible",
    
    # Should be TOXIC
    "You are an idiot",
    "I hate you so much",
    "Go kill yourself",
    "You are worthless garbage",
    
    # Should be NON-TOXIC (reporting)
    "He said you are an idiot",
    "The article discusses hate speech",
    "Someone wrote 'go die' in the comments",
    "The report mentions racist slurs",
]

print(f"{'Text':<50} {'Raw':<8} {'Report':<8} {'Adj':<8} {'Toxic?'}")
print("-"*80)

for text in test_cases:
    result = predict_toxicity(model, tokenizer, text)
    print(f"{text[:48]:<50} {result['raw_toxicity']:.2f}    {result['reporting_score']:.2f}    {result['adjusted_toxicity']:.2f}    {'❌ TOXIC' if result['is_toxic'] else '✅ OK'}")